# Dataset
[Credit Card Approval Prediction](https://www.kaggle.com/datasets/rikdifos/credit-card-approval-prediction)

## Origen
`Don't ask me where this data come from, the answer is I don't know!`

## Justificacion del problema de clasificacion
 
## Descripción del dataset

## Exploracion de datos

In [ ]:
import pandas as pd

df = pd.read_csv('datos/credit_card.csv')
y = pd.read_csv('datos/credit_card_label.csv', usecols=["label"])

# A partir de los atributos numericos continuos

## Modelo de Regresión Logistica para clasificación

##  Modelo de Arboles de Decisión.

# A partir de tanto los atributos numericos continuos como los atributos categoricos

## Modelo de Regresión Logistica para clasificación

##  Modelo de Arboles de Decisión.